# CUNY Fall 2021 Academic Calendar Scraper

This notebook scrapes the CUNY Fall 2021 academic calendar from the CCNY registrar website and creates a pandas DataFrame with the calendar data.

**Objective**: Create a DataFrame with:
- Index: Python date objects
- Column 'dow': Day of the week
- Column 'text': Event description


## Import Required Libraries


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re


## Scrape the Calendar Data


In [2]:
# URL of the CUNY Fall 2021 academic calendar
url = "https://www.ccny.cuny.edu/registrar/fall"

# Send GET request to the website
response = requests.get(url)
print(f"Status Code: {response.status_code}")

# Check if request was successful
if response.status_code == 200:
    print("Successfully retrieved the webpage!")
else:
    print(f"Failed to retrieve webpage. Status code: {response.status_code}")


Status Code: 200
Successfully retrieved the webpage!
